<a href="https://colab.research.google.com/github/sergiopdl/sPCI_sLZc/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#READ ME

Not necessarily intended for public use. Please contact jponcedeleon@ucmerced.edu with any questions or comments.

1.   Create the following directories in the root of your Google Drive (replacing "Example" with any desired name of your project):
*  /Example/Data/Inputs/Activations/
*  /Example/Data/Inputs/Events/
*  /Example/Data/Outputs/Timepoints/
*  /Example/Data/Outputs/Measures/
*  /Example/Data/Outputs/Measures/Filtered/

2.   Put your 2x2 activation matrix CSVs (channels x timepoints, 1 file per subject) in /Example/Data/Inputs/Activations/.
3.   Put your corresponding EventList TXTs (1 per subject) in /Example/Data/Inputs/Events/.
4.   Run each code chunk from top to bottom (PCI code, Python imports, and helper functions) until you get to Main.
5.  In the first code chunk in Main, configure the parameters accordingly.
6.   Run the second code in chunk Main, which outputs one file for each subject with PCI and LZc in /Example/Data/Outputs/Measures/Filtered/.
7.   Run the last code chunk to aggregate the outputs across all subjects into one file for analysis in R.







#Library installations

In [ ]:
!pip install lempel_ziv_complexity
from lempel_ziv_complexity import lempel_ziv_complexity

In [ ]:
!pip install git+https://github.com/renzocom/PCIst.git
from PCIst import pci_st

# Python imports

In [ ]:
import numpy as np
from numpy import linalg
from scipy.signal import hilbert
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import sys
import statistics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Helper functions


## Helper function to compute LZc

In [ ]:
def compute_LZc(np_signal_windowed, baseline_end_absolute, response_start_absolute):
  """
  Computes LZc for a single trial based on a baseline and response window;
    returns the result.

  Args:
    np_signal_windowed (2-d np array of floats): EEG activation values (channels x samples) windowed around a given event.
    baseline_end_absolute (int): The absolute (positive) timepoint when baseline periods end, eg 90 for 350ms baselines and a 256 sample rate.
    response_start_absolute (int): The absolute (positive) timepoint when response periods start, eg 104 for 350ms baselines (+50ms buffers) and a 256 sample rate.

  Returns:
    (float): LZc for a single trial.
  """

  # Create empty matrix for binarized data
  ncol = len(np_signal_windowed[0, response_start_absolute:])                   # For computing LZc on response data
  binarized_matrix = np.empty((0, ncol))
  lzc_per_channel = []                                                          # For computing LZc per channel (and taking the average)

  # Iterate all channels in EEG data
  for channel, signal in enumerate(np_signal_windowed):
    # EEG values are binarized using the (mean of the) instantaneous amplitude (absolute value) of the analytical (Hilbert-transformed) signal
    # per https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0133532

    # Start by hilbert-transforming the EEG data to get the analytical values
    hilbert_values_baseline = hilbert(signal[:baseline_end_absolute])           # For binarizing LZc on baseline data
    hilbert_values_response = hilbert(signal[response_start_absolute:])         # For computing LZc on response data

    # Find the instantaneous amplitude by taking the absolute value of the analytic values
    instantaneous_amplitude_values_baseline = np.abs(hilbert_values_baseline)   # For binarizing LZc on baseline data
    instantaneous_amplitude_values_response = np.abs(hilbert_values_response)   # For computing LZc on response data

    # Take the mean of the instantaneous amplitude
    mean_amplitude = np.mean(instantaneous_amplitude_values_baseline)

    # Binarize the instantaneous amplitude values based on the mean
    binarized_values = np.where(instantaneous_amplitude_values_response > mean_amplitude, 1, 0)

    # Add this binarized channel to the final binarized matrix
    binarized_matrix = np.vstack([binarized_matrix, binarized_values])

    # Compute LZc per channel
    flattened_string = "".join(binarized_values.astype(str))                    # Equivalent to .join(map(str, arr))
    lzc_per_channel.append(lempel_ziv_complexity(flattened_string))

  # Flatten the matrix timepoint by timepoint ('F' for column-major), for conversion to string for LZc calculation
  flattened_matrix_time = binarized_matrix.flatten('F')

  # Flatten the matrix channel by channel, for conversion to string for LZc calculation
  flattened_matrix_space = binarized_matrix.flatten()

  # Convert the flattened matrix to a binary string for LZc calculation
  binary_string_time = "".join(flattened_matrix_time.astype(int).flatten().astype(str))   # The order of astype() and flatten() can matter for non-binary values (per ChatGPT)
  binary_string_space = "".join(flattened_matrix_space.astype(int).flatten().astype(str))   # The order of astype() and flatten() can matter for non-binary values (per ChatGPT)

  # Calculate LZc
  lzct = lempel_ziv_complexity(binary_string_time)                                     # Alternative LZc implementation, but need to convert to single value: https://rosettacode.org/wiki/LZW_compression#Python
  lzcs = lempel_ziv_complexity(binary_string_space)

  # Return all 3 LZc values
  return (lzct, lzcs, lzc_per_channel)

## Helper function to compute PCI

In [ ]:
def compute_PCI(row, np_signal, baseline_start_factor, baseline_buffer, response_start_factor,
                response_end_factor, snr_param, k_param):
  """
  Computes PCI for a single trial based on a baseline and response window;
    returns the result.

  Args:
    row (dictionary): EventList and Timepoint data for a single trial.
    np_signal (2-d np array of floats): All EEG activation values (channels x samples) per subject.
    baseline_start_factor (int): The event-relative timepoint when baseline periods start, eg -103 for 400ms baselines and a 256 sample rate.
    baseline_buffer (int): Number of samples for buffer between baseline_end and response_start, eg 13 for 50ms and a 256 sample rate.
    response_start_factor: The event-relative timepoint when response periods start, eg typically 0, unless dividing the response into multiple windows.
    response_end_factor (int): The event-relative timepoint when response periods end, eg 103 for 400ms responses and a 256 sample rate.
    snr_param (float): Selects principal components with a signal-to-noise ratio (SNR) > min_snr (for PCI).
    k_param (float > 1): PCI noise control parameter.

  Returns:
    (float): PCI for a single trial.
  """

  # Set parameters for PCI calculation
  # To offset the response window from the baseline window by 1 timepoint
  alignment_buffer = 1

  # Compute the absolute (positive) boundaries of the baseline and response windows for the PCI par values
  baseline_end_absolute = int((baseline_start_factor * -alignment_buffer) - baseline_buffer)
  response_start_absolute = int(baseline_end_absolute + baseline_buffer + alignment_buffer + response_start_factor)
  response_end_absolute = int(baseline_end_absolute + baseline_buffer + alignment_buffer + response_end_factor)

  par = {'baseline_window':(0, baseline_end_absolute), 'response_window':(response_start_absolute, response_end_absolute),
         'k': k_param, 'min_snr': snr_param, 'max_var': 99, 'embed': False, 'n_steps': 100}

  # Pull out the actual window start and end timepoints from the eventlist data
  baseline_start_timepoint = math.floor(row['baseline_start_timepoint'])
  response_end_timepoint = math.floor(row['response_end_timepoint'])

  # Create an np array for the activation data from all channels for just the corresponding window
  np_signal_windowed = np_signal[:, baseline_start_timepoint:response_end_timepoint]

  # Create an np array for just the corresponding timepoints for the PCI calculation
  np_timepoints = np.arange(np_signal_windowed.shape[1])

  # Calculate PCI
  return (pci_st.calc_PCIst(np_signal_windowed, np_timepoints, **par),
          np_signal_windowed, par, baseline_end_absolute, baseline_buffer, response_start_absolute,
          response_end_absolute)

## Helper function to compute both measures (PCI and LZc)

In [ ]:
def compute_measures(df_onset_times_binned_timepoints, np_signal, file, baseline_start_factor,
                     baseline_buffer, response_start_factor, response_end_factor,
                     output_path_measures, input_output_path_measures_filtered, this_subject_string,
                     snr_param, k_param, n_channels, dataset, sample_rate, session):
  """
  Computes PCI and LZc for all trials (per subject);
    saves the results (CSVs) to Google Drive.

  Args:
    df_onset_times_binned_timepoints (df): EventList and timepoint data for all trials (per subject).
    np_signal (2-d np array of floats): All EEG activation values (channels x samples) per subject (for windowing).
    file (string): File name of the current file.
    baseline_start_factor (int): The event-relative timepoint when baseline periods start, eg -103 for 400ms baselines and a 256 sample rate.
        Passed to computePCI().
    baseline_buffer (int): Number of samples for buffer between baseline_end and response_start, eg 13 for 50ms and a 256 sample rate.
        Passed to compute_PCI().
    response_start_factor: The event-relative timepoint when response periods start, eg typically 0, unless dividing the response into multiple windows.
        Passed to compute_PCI().
    response_end_factor (int): The event-relative timepoint when response periods end, eg 103 for 400ms responses and a 256 sample rate.
        Passed to computePCI().
    output_path_measures (string): Google Drive path for outputting PCI/LZc results.
    input_output_path_measures_filtered (string): Google Drive path for outputting results filtered to core columns.
    this_subject_string (string): ID of current subject.
    snr_param (float): Selects principal components with a signal-to-noise ratio (SNR) > min_snr (for PCI).
        Passed to computePCI().
    k_param (float > 1): PCI noise control parameter.
        Passed to computePCI().
    n_channels (int): Number of EEG channels specified (for output filename).
    dataset: Name of dataset (for output filename).
    sample_rate (int): EEG sample rate (for timepoint conversion).
    session (string): Session number for filename outputs.
  """

  # Create new df for final PCI/LZc results
  df_measures = df_onset_times_binned_timepoints
  df_measures["lzc_per_channel"] = ""                                           # Initialize column for lzc_per_channel to type string

  # Loop through all events in the eventlist data
  for event, row in df_onset_times_binned_timepoints.iterrows():

    # Compute PCI
    (pci, np_signal_windowed, par,
     baseline_end_absolute, baseline_buffer,
     response_start_absolute, response_end_absolute) = compute_PCI(row, np_signal,
                                                                   baseline_start_factor,
                                                                   baseline_buffer,
                                                                   response_start_factor,
                                                                   response_end_factor,
                                                                   snr_param, k_param)

    # Skip first row if start window goes negative, or last row if end window goes beyond available data
    if (np_signal_windowed.size == 0) or (np_signal_windowed[0].size < -baseline_start_factor + response_end_factor):
      print("WARNING: PCI/LZc WINDOW", -baseline_start_factor + response_end_factor, "LARGER THAN SIGNAL", np_signal_windowed[0].size) # debug
      print("SETTING PCI AND LZC TO 0 AND CONTINUING TO NEXT ROW/TRIAL/SUBJECT")
      df_measures.loc[event, "pci"] = 0
      df_measures.loc[event, "lzct"] = 0
      df_measures.loc[event, "lzcs"] = 0
      df_measures.loc[event, "lzc_per_channel"] = 0

    else:

      # Add the PCI values to the results df
      df_measures.loc[event, "pci"] = pci

      # Compute LZc
      lzct, lzcs, lzc_per_channel = compute_LZc(np_signal_windowed, baseline_end_absolute, response_start_absolute)

      # Add the LZc values to the results df
      df_measures.loc[event, "lzct"] = lzct
      df_measures.loc[event, "lzcs"] = lzcs
      df_measures.loc[event, "lzc_per_channel"] = ",".join(map(str, lzc_per_channel))

  # Save new dfs with PCI/LZc results to Google Drive
  df_measures.to_csv(output_path_measures + dataset + "_" + this_subject_string + session +
                     "_chann" + str(n_channels) +
                     "_samprate" + str(sample_rate) +
                     "_base0-" + str(baseline_end_absolute) +
                     "_resp" + str(response_start_absolute) + "-" + str(response_end_absolute) +
                     "_timepoints_pci_lzc.csv")

  df_measures_filtered = df_measures[["ecode", "a_flags", "bin", "pci", "lzct", "lzcs", "lzc_per_channel"]]
  df_measures_filtered.to_csv(input_output_path_measures_filtered + dataset + "_" + this_subject_string + session +
                              "_chann" + str(n_channels) +
                              "_samprate" + str(sample_rate) +
                              "_base0-" + str(baseline_end_absolute) +
                              "_resp" + str(response_start_absolute) + "-" + str(response_end_absolute) +
                              "_ecode_aflags_bin_pci_lzc.csv")

## Compute timepoints/windows

In [ ]:
def compute_timepoints(df_onset_times_binned, input_output_path_timepoints, sample_rate,
                       baseline_start_factor, baseline_buffer, response_start_factor, response_end_factor,
                       file, n_channels, this_subject_string, session):
  """
  Converts trial onset times to timepoints (per subject);
    computes baseline start and response end timepoints;
    saves new dfs to Google Drive.

  Args:
    df_onset_times_binned (df): EventList data for all trials (per subject).
    input_output_path_timepoints (string): Google Drive path for outputting timepoints.
    sample_rate (int): EEG sample rate (for timepoint conversion).
    baseline_start_factor (int): The event-relative timepoint when baseline periods start, eg -103 for 400ms baselines and a 256 sample rate.
    baseline_buffer (int): Number of samples for buffer between baseline_end and response_start, eg 13 for 50ms and a 256 sample rate.
    response_end_factor (int): The event-relative timepoint when response periods end, eg 103 for 400ms responses and a 256 sample rate.
    response_start_factor: The event-relative timepoint when response periods start, eg typically 0, unless dividing the response into multiple windows.
    file (string): File name of the current file.
    n_channels (int): Number of EEG channels specified (for output filename).
    this_subject_string (string): ID of current subject.
    session (string): Session number for filename outputs.

  Returns:
    df_onset_times_binned_timepoints (df): EventList and timepoint data for all trials (per subject).
  """

  # Add new columns for timepoints and windows
  df_onset_times_binned_timepoints = df_onset_times_binned.copy()
  df_onset_times_binned_timepoints['timepoint'] = df_onset_times_binned_timepoints['onset'].apply(lambda x: x * sample_rate)
  df_onset_times_binned_timepoints['baseline_start_timepoint'] = df_onset_times_binned_timepoints['onset'].apply(lambda x: x * sample_rate + baseline_start_factor)
  df_onset_times_binned_timepoints['response_end_timepoint'] = df_onset_times_binned_timepoints['onset'].apply(lambda x: x * sample_rate + response_end_factor)

  # Add sanity check that response_end_timepoint ends before next baseline_start_timepoint?

  # Save new df with timepoint/window data to Google Drive
  df_onset_times_binned_timepoints.to_csv(input_output_path_timepoints + dataset + "_" + this_subject_string + session +
                                          "_chann" + str(n_channels) +
                                          "_base" + str(baseline_start_factor) + "_-" + str(baseline_buffer) +
                                          "_resp" + str(response_start_factor) + "-" + str(response_end_factor) +
                                          "_timepoints.csv")

  return df_onset_times_binned_timepoints

## Convert eventlist data

In [ ]:
def convert_eventlist_data(dataset, input_path_events, eventlist_files_sorted, this_subject_idx, ignore_rows):
  """
  Converts EventList data from .txt to .csv (including bins).

  Args:
    dataset: Name of dataset (for output filename).
    input_path_events (string): Google Drive path for inputting EventList data (for all subjects).
    eventlist_files_sorted (list of strings): List of EventList filenames, sorted increasing to match subject IDs.
    this_subject_idx (int): Index of current subject.
    ignore_rows (rows): Number of rows to ignore from EventList.txt files.

  Returns:
    df_onset_times_binned (df): EventList data with bins for all trials (per subject).
  """

  df_onset_times = pd.read_csv(input_path_events + eventlist_files_sorted[this_subject_idx],
                               skiprows=ignore_rows, sep=r'\s+(?![^\[]*\])' ,   # split on spaces only when not within brackets, per GPT # sep='\s+' # delim_whitespace=True deprecated
                               header=None,                                     # Ignore header/column names due to formatting inconsistencies
                               engine='python')                                 # Specify the python engine because the c engine does not support regex separators

  # Remove erroneous column and add correct column names back to the df
  df_onset_times.drop(df_onset_times.columns[[9]], axis=1, inplace=True)

  df_onset_times.columns =['item', 'bepoch', 'ecode', 'label', 'onset', 'diff', 'dura',
                           'b_flags', 'a_flags', 'enable']

  """
  Convert "enable" column to bins
    Ensure x is a string before using endswith(),
    strip the square brackets,
    split the resulting string (by spaces),
    only if the resulting string isn't empty,
    choose the last list item (else set bin to 0)
  """
  df_onset_times_binned = df_onset_times
  df_onset_times_binned['bin'] = df_onset_times_binned['enable'].apply(lambda x: 1 if str(x).strip('[]').split() and str(x).strip('[]').split()[-1].endswith('1')
                                                          else (2 if str(x).strip('[]').split() and str(x).strip('[]').split()[-1].endswith('2')
                                                          else (3 if str(x).strip('[]').split() and str(x).strip('[]').split()[-1].endswith('3')
                                                          else (4 if str(x).strip('[]').split() and str(x).strip('[]').split()[-1].endswith('4')
                                                          else (5 if str(x).strip('[]').split() and str(x).strip('[]').split()[-1].endswith('5')
                                                          else (6 if str(x).strip('[]').split() and str(x).strip('[]').split()[-1].endswith('6')
                                                          else (7 if str(x).strip('[]').split() and str(x).strip('[]').split()[-1].endswith('7')
                                                          else (8 if str(x).strip('[]').split() and str(x).strip('[]').split()[-1].endswith('8')
                                                          else 0))))))))

  return df_onset_times_binned

## Run subjects

In [ ]:
def run_subjects(activation_files_sorted, input_path_activations, dataset, input_path_events,
                 eventlist_files_sorted, input_output_path_timepoints, output_path_measures,
                 input_output_path_measures_filtered, output_path_general, baseline_start_factor,
                 baseline_buffer, response_start_factor, response_end_factor, snr_param,
                 k_param, timepoint_files_sorted, measures_filtered_files_sorted, subject_ids,
                 sample_rate, n_channels, ignore_rows, session_in_filename):
  """
  Loops through all subjects;
    Reads in activation data;
    Converts corresponding EventList data;
    Computes corresponding timepoints (for PCI/LZc);
    Computes PCI/LZC;
    Outputs metadata.

  Args:
    activation_files_sorted (list of strings): List of EEG activation filenames, sorted increasing to match subject IDs.
    input_path_activations (string): Google Drive path for inputting Acivations data (for all subjects)
    dataset: Name of dataset (for output filename). Passed to convert_eventlist_data().
    input_path_events (string): Google Drive path for inputting EventList data (for all subjects).
        Passed to convert_eventlist_data().
    eventlist_files_sorted (list of strings): List of EventList filenames, sorted increasing to match subject IDs.
        Passed to convert_eventlist_data().
    input_output_path_timepoints (string): Google Drive path for outputting timepoints.
        Passed to compute_timepoints().
    output_path_measures (string): Google Drive path for outputting PCI/LZc results.
        Passed to compute_measures().
    input_output_path_measures_filtered (string): Google Drive path for outputting results filtered to core columns.
        Passed to compute_measures().
    output_path_general (string): Google Drive path for outputting metadata file.
    baseline_start_factor (int): The event-relative timepoint when baseline periods start, eg -103 for 400ms baselines and a 256 sample rate.
        Passed to compute_timepoints().
    baseline_buffer (int): Number of samples for buffer between baseline_end and response_start, eg 13 for 50ms and a 256 sample rate.
        Passed to compute_timepoints().
    response_start_factor: The event-relative timepoint when response periods start, eg typically 0, unless dividing the response into multiple windows.
        Passed to compute_timepoints().
    response_end_factor (int): The event-relative timepoint when response periods end, eg 103 for 400ms responses and a 256 sample rate.
        Passed to compute_timepoints().
    snr_param (float): Selects principal components with a signal-to-noise ratio (SNR) > min_snr.
        Passed to compute_measures().
    k_param (float > 1): Noise control parameter. Passed to compute_measures().
    timepoint_files_sorted (list of strings): List of timepoint filenames, sorted increasing to match subject IDs.
        (If > 0, doesn't re-run convert_eventlist() and compute_timepoints()).
    measures_filtered_files_sorted (list of strings):  List of timepoint filenames, sorted increasing to match subject IDs.
        (If > 0, doesn't re-run compute_measures().
    subject_ids (string): List of subject IDs.
    sample_rate (int): EEG sample rate (for timepoint conversion).
        Passed to compute_timepoints() and compute_measures().
    n_channels (int): Number of EEG channels specified (for output filename).
        Passed to compute_timepoints().
    session_in_filename: If session number is in filename.

  Returns:
    (no value): Script completes.
  """

  # For activation-data sanity check and onset-time conversion
  seconds_per_minute = 60

  # Create lists for metadata
  n_subjects_list = []
  n_samples = []
  n_seconds = []
  n_minutes = []

  # Loop through all activation/eventlist files (one per subject)
  for this_subject_idx, file in enumerate(activation_files_sorted):

    print()
    print(file)

    # Extract subject ID from filename
    this_subject_string = file.partition("_")[0]

    # Check that this subject is in the list of IDs to process
    if this_subject_string not in subject_ids:
      print("SKIPPING SUBJECT:", this_subject_string) # debug
      continue

    # Save subject ID for outputting to filenames
    n_subjects_list.append(this_subject_string)

    # Read in activation data for one subject
    df_activations = pd.read_csv(input_path_activations + file, header=None)

    # Convert activation data to numpy array for PCI/LZc calculation
    np_signal = df_activations.iloc[:n_channels, :].to_numpy()
    print("SHAPE (CHANNELS X SAMPLES):", np_signal.shape) # debug
    n_samples.append(np_signal.shape[1])

    # Sanity check that there are approximately # minutes worth of data
    n_seconds.append(np_signal.shape[1] / sample_rate)
    n_minutes.append(np_signal.shape[1] / sample_rate / seconds_per_minute)
    print("MINUTES OF DATA:", np_signal.shape[1] / sample_rate / seconds_per_minute) # debug

    # Check for session number in filename
    if (session_in_filename):
      session = "_" + file.split("_")[1]
    else:
      session = ""

    # Check for existing timepoint data
    if(len(timepoint_files_sorted) > 0): # REFACTOR TO CHECK FOR ACTUAL ENTRY
      df_onset_times_binned_timepoints = pd.read_csv(input_output_path_timepoints + timepoint_files_sorted[this_subject_idx])

    else:
      # Convert eventlist data
      df_onset_times_binned = convert_eventlist_data(dataset, input_path_events, eventlist_files_sorted, this_subject_idx, ignore_rows)

      # Compute timepoints/windows
      df_onset_times_binned_timepoints = compute_timepoints(df_onset_times_binned,
                                                            input_output_path_timepoints,
                                                            sample_rate, baseline_start_factor,
                                                            baseline_buffer, response_start_factor,
                                                            response_end_factor, file,
                                                            n_channels, this_subject_string,
                                                            session)

    # Add check for existing PCI and LZc data
    # Compute PCI and LZc
    compute_measures(df_onset_times_binned_timepoints, np_signal, file, baseline_start_factor,
                    baseline_buffer, response_start_factor, response_end_factor, output_path_measures,
                    input_output_path_measures_filtered, this_subject_string, snr_param, k_param,
                    n_channels, dataset, sample_rate, session)

  # Create metadata df and output to CSV
  df_metadata = pd.DataFrame({'subject_id': n_subjects_list,
                              'n_samples': n_samples,
                              'n_seconds': n_seconds,
                              'n_minutes': n_minutes})
  df_metadata.to_csv(output_path_general + dataset + "_subject_metadata.csv")

  return

## Load data/files

In [ ]:
def load_data(input_path_activations, input_path_events, input_output_path_timepoints,
              input_output_path_measures_filtered):
  """
  Loads EEG activation, Eventlist, and timepoint data from Google Drive.

  Args:
    input_path_activations (string): Google Drive path for inputting Acivations data (for all subjects)
    input_path_events (string): Google Drive path for inputting EventList data (for all subjects).
    input_output_path_timepoints (string): Google Drive path for outputting timepoints.
    input_output_path_measures_filtered (string): Google Drive path for outputting results filtered to core columns.

  Returns:
    activation_files_sorted (list of strings): List of EEG activation filenames, sorted increasing to match subject IDs.
    eventlist_files_sorted (list of strings): List of EventList filenames, sorted increasing to match subject IDs.
    timepoint_files_sorted (list of strings): List of timepoint filenames, sorted increasing to match subject IDs.
    measures_filtered_files_sorted (list of strings): List of measures filenames, sorted increasing to match subject IDs.
  """

  # Grab all activation, eventlist, and timepoint files, and sort alphabetically
  activation_files = [x for x in os.listdir(input_path_activations)]
  eventlist_files = [x for x in os.listdir(input_path_events)]
  timepoint_files = [x for x in os.listdir(input_output_path_timepoints)]
  measures_filtered_files = [x for x in os.listdir(input_output_path_measures_filtered)]

  # If the number of files don't match, stop running
  if(len(activation_files) != len(eventlist_files)):
    raise ValueError('MISMATCHING FILE NUMBERS!')

  # Check filenames for whether to sort by first digit or first+second digits
  if activation_files[0].split("_")[1].isdigit():
    activation_files_sorted = sorted(activation_files, key=lambda x: (int(x.split("_")[0]), int(x.split("_")[1])))
    eventlist_files_sorted = sorted(eventlist_files, key=lambda x: (int(x.split("_")[0]), int(x.split("_")[1])))
    timepoint_files_sorted = sorted(timepoint_files, key=lambda x: (int(x.split("_")[1]), int(x.split("_")[2])))
    measures_filtered_files_sorted = sorted(measures_filtered_files, key=lambda x: (int(x.split("_")[1]), int(x.split("_")[2])))
  else:
    activation_files_sorted = sorted(activation_files, key=lambda x: int(x.split("_")[0]))
    eventlist_files_sorted = sorted(eventlist_files, key=lambda x: int(x.split("_")[0]))
    timepoint_files_sorted = sorted(timepoint_files, key=lambda x: int(x.split("_")[0]))
    measures_filtered_files_sorted = sorted(measures_filtered_files, key=lambda x: int(x.split("_")[0]))

  return (activation_files_sorted, eventlist_files_sorted, timepoint_files_sorted, measures_filtered_files_sorted)

# Main

### Configure

In [ ]:
# Set parameters for dataset
file_path_modifier = ""                                                         # eg "Research Projects/"
dataset = "MMN"                                                                 # eg N170, P3, MMN; Ekman
session_in_filename = False                                                     # If session number is in filename, eg False for N170; True for Ekman
subject_ids = list(map(str, range(1, 41)))                                      # eg 1-41 for N170, P3, MMN; 351-389 for Ekman
n_channels = 28                                                                 # eg 28 for N170, P3, MMN; 19 for Ekman
sample_rate = 256                                                               # eg 256 for N170, P3, MMN; 512 for Ekman

# Rows to ignore in Eventlist.txt files
ignore_rows = 28                                                                # eg 30 for N170; 28 for P3, MMN; 36 Ekman

# Set PCI window parameter values for conversion to timepoints
baseline_start_ms = -293                                                        # PCIst default -400; -293 for MMN
baseline_buffer_ms = 50                                                         # PCIst default 50
response_start_ms = 0                                                           # PCI default 0
response_end_ms = 256                                                           # PCIst default 300, + 100 ms for visual sensory processing (eg P100); 256 for MMN
print("BASELINE AND RESPONSE WINDOWS IN MS:", baseline_start_ms, 0-baseline_buffer_ms, response_start_ms, response_end_ms)

# Subtract samples for the buffer between baseline end and response start
baseline_buffer = math.ceil((baseline_buffer_ms / 1000) * sample_rate)

# Convert PCI window parameter values to 0-relative timepoint factors
baseline_start_factor = math.floor(baseline_start_ms / 1000 * sample_rate)
response_start_factor = math.ceil(response_start_ms / 1000 * sample_rate)
response_end_factor = math.ceil((response_end_ms) / 1000 * sample_rate)
print("BASELINE AND RESPONSE WINDOW TIMEPOINT CONVERSION FACTORS:", baseline_start_factor, 0-baseline_buffer, response_start_factor, response_end_factor)

"""
The following parameters may be suboptimal for evoked signals with lower signal-to-noise ratio,
  such as those produced by peripheral stimulation, where it may be necessary to increase k
  and/or the minimum SNR (1.8 per Alessandra Dallavecchia) to control for stationary baseline-like activations.
"""
snr_param = 1.1                                                                 # (float) Selects principal components with a signal-to-noise ratio (SNR) > min_snr (for PCI)
k_param = 1.2                                                                   # (float > 1): PCI noise control parameter

# Set Google Drive locations for raw data files
input_path_activations = "/content/drive/MyDrive/" + file_path_modifier + dataset + "/Data/Inputs/Activations/"
input_path_events = "/content/drive/MyDrive/" + file_path_modifier + dataset + "/Data/Inputs/Events/"
input_output_path_timepoints = "/content/drive/MyDrive/" + file_path_modifier + dataset + "/Data/Outputs/Timepoints/"
output_path_measures = "/content/drive/MyDrive/" + file_path_modifier + dataset + "/Data/Outputs/Measures/"
input_output_path_measures_filtered = "/content/drive/MyDrive/" + file_path_modifier + dataset + "/Data/Outputs/Measures/Filtered/"
output_path_general = "/content/drive/MyDrive/" + file_path_modifier + dataset + "/Data/Outputs/"

# Initialize empty lists for sorted files
activation_files_sorted = []
eventlist_files_sorted = []
timepoint_files_sorted = []

# Load data
(activation_files_sorted, eventlist_files_sorted, timepoint_files_sorted, measures_filtered_files_sorted) = load_data(input_path_activations,
                                                                                                                      input_path_events,
                                                                                                                      input_output_path_timepoints,
                                                                                                                      input_output_path_measures_filtered)
print(activation_files_sorted)
print(eventlist_files_sorted)
print(timepoint_files_sorted)
print(measures_filtered_files_sorted)

### Run

In [ ]:
# Main function call
run_subjects(activation_files_sorted, input_path_activations, dataset, input_path_events,
             eventlist_files_sorted, input_output_path_timepoints, output_path_measures,
             input_output_path_measures_filtered, output_path_general, baseline_start_factor,
             baseline_buffer, response_start_factor, response_end_factor, snr_param,
             k_param, timepoint_files_sorted, measures_filtered_files_sorted, subject_ids,
             sample_rate, n_channels, ignore_rows, session_in_filename)

# Output subject-aggreated df for statistical analyses

In [ ]:
# Read back in trial-by-trial PCI results and sort by subject id
input_output_measures_filtered = "/content/drive/MyDrive/" + file_path_modifier + dataset + "/Data/Outputs/Measures/Filtered/"
output_path_general = "/content/drive/MyDrive/" + file_path_modifier + dataset + "/Data/Outputs/"
results_files = [x for x in os.listdir(input_output_measures_filtered)]
results_files_sorted = sorted(results_files, key=lambda x: int(x.split("_")[1]))

# Initialize new df for aggregate of all subjects
aggregated_df = pd.DataFrame()

# Loop through all subjects
for subject, file in enumerate(results_files_sorted):

  # Read in results, rename trial column, and add subject_id and session columns
  df_pci_filtered = pd.read_csv(input_output_measures_filtered + file)
  df_pci_filtered.rename(columns = {'Unnamed: 0':'trial'}, inplace=True)
  df_pci_filtered.rename(columns = {'bin':'condition'}, inplace=True)
  df_pci_filtered['subject_id'] = file.split("_")[1]
  if session_in_filename:
    df_pci_filtered['session'] = file.split("_")[2]
  aggregated_df = pd.concat([aggregated_df, df_pci_filtered])

# Reset a new sequential index
aggregated_df = aggregated_df.reset_index(drop=True)

# Save final aggregated csv for statistical analysis
aggregated_df.to_csv(output_path_general + dataset + "_aggregated" +
                     "_chann" + str(n_channels) +
                     "_samprate" + str(sample_rate) +
                     "_base" + str(baseline_start_factor) + "_-" + str(baseline_buffer) +
                     "_resp" + str(response_start_factor) + "-" + str(response_end_factor) +
                     "_trial_ecode_aflags_condition_pci_lzc_subjectID.csv")